# Luck-Meter Exploration

This notebook is an attempt to explore different ways of representing “luck” in Blood Bowl. "Luck" is not well-defined, and will likely mean different things dependig on the context, so the goal of this notebook is to examine several possible approaches and compare them against each other. Ideally finding a way to represent this in the Jervis FFB Client.

Most statistical analysis requires that we can assign probabilities to rolls. For some rolls, like Dodges or Rushes, this is trivial. For others, like block dice, it gets harder as it is unclear how to treat pushes, e.g., during one-turn-attempts they are probably "success" while during normal play, they are at best "neutral" (with exceptions). And finally, we have rolls like bounces that are close to impossible to assign success probabilities to.

For now, we just examine the simple case of rolling single D6 with clear failure/success probabilities.

**Caveat:** I am not a trained statistician, so it is very likely I am using some terms in this notebook imprecise or wrong. A lot of the implementations and wordings in this notebook came from either Wikipedia or ChatGPT.

### How to look at dice rolls?

When rolling dice in Blood Bowl, we are looking at two properties:
1) We want them to be fair, i.e., each value needs to roll each value roughly the same number of times (16.67%).
2) We want to roll the target number (e.g. 4+) every time. If we do, we say we are "lucky".

Some observations:
- These two properties are not connected, but generally it will not be possible to be maximally lucky using fair dice.
- The same set of rolls can result in very different "luck" values. E.g. if you use all you 6's on 2+ dodges vs. using them rolling for armor.


### Statistical analysis of dice rolls

If we are rolling _n_ D6, each with their own probability of success. The following properties are true:

* The total number of successes X is the sum of n independent but non-identical Bernoulli random variables.
* That distribution of this is the Poisson Binomial Distribution.
* For the amount of dice rolls in Blood Bowl, we expect 100-200 rolls and their probability of success is between
  0.83 and 0.16. In this case the Normal Distribution can be used as an approximation.


In [1]:
%use dataframe, kandy

In [2]:
import kotlin.random.Random

// First we setup some helpers and create the distributions we want
// to investigate. For now, we only look at single D6's

// Set to hard-coded value to reproduce results
val seed = 8033212925733483815 // Random.nextLong()
val random = Random(seed)

/**
 * Die that can be configured to unfair.
 *
 * [bias] determines the percentage a given range is selected
 * [towardsLow] determines if [bias] is applied to 1-3 (on D6) or 4-6 (on D6)
 */
fun skewedDie(
    sides: Int = 6,
    bias: Double = 0.5,
    towardsLow: Boolean = true
): Int {
    require(sides >= 2) { "Die must have at least 2 sides" }
    require(bias in 0.0..1.0) { "Bias must be between 0.0 and 1.0" }

    val half = sides / 2
    val lowCount = half + if (sides % 2 != 0) 1 else 0
    val highCount = sides - lowCount

    val (favoredCount, unfavoredCount) =
        if (towardsLow) lowCount to highCount else highCount to lowCount

    val pFav = bias / favoredCount
    val pUnfav = (1 - bias) / unfavoredCount

    val r = random.nextDouble()
    var acc = 0.0

    if (towardsLow) {
        for (i in 1..lowCount) {
            acc += pFav
            if (r < acc) return i
        }
        for (i in (lowCount + 1)..sides) {
            acc += pUnfav
            if (r < acc) return i
        }
    } else {
        for (i in 1..lowCount) {
            acc += pUnfav
            if (r < acc) return i
        }
        for (i in (lowCount + 1)..sides) {
            acc += pFav
            if (r < acc) return i
        }
    }
    return sides // fallback due to rounding
}

@DataSchema
data class DiceRoll(val roll: Int, val target: Int, val sides: Int) {
    fun successPropability(): Double {
        return (sides - (target - 1)) / sides.toDouble()
    }
    fun failurePropability(): Double {
        if (target > sides) return 1.0
        return (target - 1) / sides.toDouble()
    }
    fun neturalPropability() = 0.0
    fun isSuccess() = roll >= target
    fun isFailure() = roll < target
}

In [3]:
// Create the distributions we want to test.
// Make sure the amount of rolls are divisible by 6, as it makes it easier
// to compare results later.
val rolls: Int = 20 * 6
val d6BucketSize: Int = rolls / 6

// All dice roll the same value
val allOnes = (1..rolls).map {
    DiceRoll(1, random.nextInt(2, 7), 6)
}
val allSixes = (1..rolls).map {
    DiceRoll(6, random.nextInt(2, 7), 6)
}

// Even buckets, all fail
val fairDiceAllFail = (1..rolls).map { n ->
    val roll = ((n - 1) / d6BucketSize) + 1 // Set dice value based on "bucket
    DiceRoll(roll, 7, 6)
}
val fairDiceAllSucceed = (1..rolls).map { n ->
    val roll = ((n - 1) / d6BucketSize) + 1 // Set dice value based on "bucket
    DiceRoll(roll, 1, 6)
}

// Even distribution of dice. Target roll of 4 (=50% success)
val fairDiceHalfSucceed = (1..rolls).map { n ->
    val roll = ((n - 1) / d6BucketSize) + 1 // Set dice value based on "bucket
    DiceRoll(roll, 4, 6)
}

// Distribution skewed with 75% towards the bottom
val skewedTowardsBottomHalf = (1..rolls).map { n ->
    val roll = skewedDie(sides = 6, bias = 0.75, towardsLow = true)
    val target = random.nextInt(2, 7)
    DiceRoll(roll, target, 6)
}

// Distribution skewed with 75% towards the top
val skewedTowardsTopHalf = (1..rolls).map { n ->
    val roll = skewedDie(sides = 6, bias = 0.75, towardsLow = false)
    val target = random.nextInt(2, 7)
    DiceRoll(roll, target, 6)
}

// Random roll and target between 2-6
val random1 = (1..rolls).map { n ->
    val roll = random.nextInt(1, 7)
    val target = random.nextInt(2, 7)
    DiceRoll(roll, target, 6)
}

// Random roll and target between 2-6
val random2 = (1..rolls).map { n ->
    val roll = random.nextInt(1, 7)
    val target = random.nextInt(2, 7)
    DiceRoll(roll, target, 6)
}

// Random roll and target between 2-6
val random3 = (1..rolls).map { n ->
    val roll = random.nextInt(1, 7)
    val target = random.nextInt(2, 7)
    DiceRoll(roll, target, 6)
}

// Random roll with target 4+
val randomDiceWith4Target = (1..rolls).map {
    val roll = random.nextInt(1, 7)
    DiceRoll(roll, 4, 6)
}

val distributions = listOf(
    "All 1's" to allOnes,
    "All 6's" to allSixes,
    "Even buckets - All fail" to fairDiceAllFail,
    "Even buckets - All succed" to fairDiceAllSucceed,
    "Even buckets - Half succeed" to fairDiceHalfSucceed,
    "Scewed towards bottom" to skewedTowardsBottomHalf,
    "Scewed towards top" to skewedTowardsTopHalf,
    "Random (4+)" to randomDiceWith4Target,
    "Random 1"  to random1,
    "Random 2" to random2,
    "Random 3" to random3
)

In [4]:
// Bucket rolls (how many of each dice was rolled).
// For fair dice we expect 20 in each bucket.
val rollDistributions = distributions.map {
    val title = it.first
    val df = it.second.toDataFrame()
    title to df.groupBy { roll }.count()
}.let { dfs ->
    // ensure full dice set {1..6}, fill missing with 0
    dfs.map { (name, df) ->
        (1..6).toDataFrame("roll")
            .leftJoin(df) { "roll" match "roll" }
            .fillNA("count").withZero()
            .rename("count").into(name)
    }.reduce { acc, df ->
        acc.join(df) { "roll" match "roll" }
    }
}.gather { allExcept("roll") }
    .into("distribution", "value")
    .groupBy("distribution")
    .pivot("roll")
    .aggregate { first().get("value") }
    .flatten()
rollDistributions

distribution,1,2,3,4,5,6
All 1's,120,0,0,0,0,0
All 6's,0,0,0,0,0,120
Even buckets - All fail,20,20,20,20,20,20
Even buckets - All succed,20,20,20,20,20,20
Even buckets - Half succeed,20,20,20,20,20,20
Scewed towards bottom,28,37,26,9,10,10
Scewed towards top,9,10,10,22,41,28
Random (4+),22,16,23,21,19,19
Random 1,27,16,22,15,20,20
Random 2,20,15,19,24,23,19


In [5]:
// Plot roll distribution.
// Ignore distibutions that are either all 1's or all 6's as it skew the result too much.
val bucketData = rollDistributions
    .getRows(2..rollDistributions.rowsCount() - 1)
    .gather { colsOf<Int>() }
    .into("roll", "count")

// Plot how many of each roll to get a sense of how fair they roll
plot(bucketData) {
    bars {
        x("roll")
        y("count")
        fillColor("distribution")
        position = Position.dodge()
    }
    hLine {
        yIntercept.constant(20)
        color = Color.WHITE
        width = 1.0
        alpha = 0.8
        type = LineType.DASHED
    }

    x.axis.name = "Roll"
    y.axis.name = "Count"
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.5.1/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="iuJLvj"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 
 (function() {
 // ----------
 
 var plotSpec={
"mapping":{
},
"data":{
"roll":["1","2","3","4","5","6","1","2","3","4","5","6","1","2","3","4","5","6","1","2","3","4","5","6","1","2","3","4","5","6","1","2","3","4","5","6","1","2","3","4","5","6","1","2","3","4","5","6","1","2","3","4","5","6"],
"count":[20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,20.0,28.0,37.0,26.0,9.0,10.0,10.0,9.0,10.0,10.0,22.0,41.0,28.0,22.0,16.0,23.0,21.0,19.0,19.0,27.0,16.0,22.0,15.0,20.0,20.0,20.0,15.0,19.0,24.0,23.0,19.0,19.0,16.0,18.0,26.0,18.0,23.0],
"distribution":["Even buckets - All fail","Even buckets - All fail","Even buckets - All fail","Even buckets - All fail","Even buckets - All fail","Even buckets - All fail","Even buckets - All succed","Even buckets - All succed","Even buckets - All succed","Even buckets - All succed","Even buckets - All succed","Even buckets - All succed","Even buckets - Half succeed","Even buckets - Half succeed","Even buckets - Half succeed","Even buckets - Half succeed","Even buckets - Half succeed","Even buckets - Half succeed","Scewed towards bottom","Scewed towards bottom","Scewed towards bottom","Scewed towards bottom","Scewed towards bottom","Scewed towards bottom","Scewed towards top","Scewed towards top","Scewed towards top","Scewed towards top","Scewed towards top","Scewed towards top","Random (4+)","Random (4+)","Random (4+)","Random (4+)","Random (4+)","Random (4+)","Random 1","Random 1","Random 1","Random 1","Random 1","Random 1","Random 2","Random 2","Random 2","Random 2","Random 2","Random 2","Random 3","Random 3","Random 3","Random 3","Random 3","Random 3"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"fill",
"discrete":true
},{
"aesthetic":"x",
"name":"Roll",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Count",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"roll",
"y":"count",
"fill":"distribution"
},
"stat":"identity",
"sampling":"none",
"inherit_aes":false,
"position":"dodge",
"geom":"bar",
"data":{
}
},{
"mapping":{
},
"stat":"identity",
"yintercept":20.0,
"color":"#ffffff",
"size":1.0,
"linetype":"dashed",
"sampling":"none",
"alpha":0.8,
"inherit_aes":false,
"position":"identity",
"geom":"hline",
"data":{
}
}],
"data_meta":{
"series_annotations":[{
"type":"str",
"column":"roll"
},{
"type":"int",
"column":"count"
},{
"type":"str",
"column":"distribution"
}]
},
"spec_id":"2"
};
 var containerDiv = document.getElementById("iuJLvj");
 
 var toolbar = null;
 var plotContainer = containerDiv; 
 
 var options = {
 sizing: {
 width_mode: "fixed",
 height_mode: "fixed",
 width: 600.0,
 height: 400.0
 }
 };
 var fig = LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer, options);
 if (toolbar) {
 toolbar.bind(fig);
 }
 
 // ----------
 })();
 
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 
 
 
 
 2 
 
 
 
 
 
 
 
 
 3 
 
 
 
 
 
 
 
 
 4 
 
 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 
 
 6 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 5 
 
 
 
 
 
 
 10 
 
 
 
 
 
 
 15 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 25 
 
 
 
 
 
 
 30 
 
 
 
 
 
 
 35 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 
 
 Count 
 
 
 
 
 Roll 
 
 
 
 
 
 
 
 
 distribution 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Even buckets 

In [6]:
/**
 * Jensen–Shannon divergence (JSD)
 * Informally, this is the "distance" [0-1] away from the expected distribution.
 *
 * Critic:
 * - While the JSD value tells you something about how "fair" a die is, it doesn't
 *   say anything about "luck",.e.g., rolling all 1's and rolling all 6's will
 *   result in the same JSD value.
 * - It turns out the maximum value is ~0.65486. Apparently because bigger values
 *   can only happen when the distributions have disjoint support. I didn't totally
 *   understand ChatGPTs answer here with regard to dice rolls, but I guess we can
 *   scale the values so they fit in the [0-1] range if needed.
 * - JSD doesn't scale linearly, so exposing it as 0-100% is wrong. If exposed it
 *   should be a "fairness index" instead.
 *
 * See https://en.wikipedia.org/wiki/Jensen%E2%80%93Shannon_divergence
 */

/**
 * Calculate the Jensen-Shanon Divergence value.
 *
 * @param rollProbabilities The result of all dice rolls with each index + 1
 * being a side on the die and the probability for rolling that die in the trial.
 */
fun jensenShannonBits(rollProbabilities: List<Double>): Double {
    val sides = rollProbabilities.size
    val u = 1 / sides.toDouble() // Uniform distribution
    var jsd = 0.0
    for (i in 1..sides) {
        val p = rollProbabilities[i - 1]
        val m = 0.5 * (p + u)
        if (p > 0.0) {
            jsd += 0.5 * p * (ln(p / m) / ln(2.0))
        }
        jsd += 0.5 * u * (ln(u / m) / ln(2.0))
    }
    return jsd // ∈ [0, 1] (bounded by 1 bit)
}

val jsdMaxForD6 = 0.6548575458
val jsdScale = 1.0 / jsdMaxForD6
val jsdValues = rollDistributions
     // Informally try to use the numeric "distance" from the expected value as a heuristic.
     // Since this will require another value to also shift, we just count values above the expected,
     // so we do not "double-count".
    .add("distance") { row ->
        row.valuesOf<Int>().fold(0) { acc, v ->
            if (v > d6BucketSize) {
                acc + (v - d6BucketSize)
            } else {
                acc
            }
        }
    }
    .convert { allExcept("distribution", "distance") }.toDouble()
    .update { colsOf<Double>() }.with { it / rolls }
    .groupBy("distribution")
    .aggregate {
        val rollProps = it.values { colsOf<Double>() }.toList()
        val jsd = jensenShannonBits(rollProps)
        // Convert into a [0-1] range with 1.0 meaning uniform distribution
        jsd into "jsd"
        abs(jsdMaxForD6 - jsd)*jsdScale into "adjustedJSD"
        it.values().last() into "distance"
    }
jsdValues

distribution,jsd,adjustedJSD,distance
All 1's,0.654858,0.000000,100
All 6's,0.654858,0.000000,100
Even buckets - All fail,0.000000,1.000000,0
Even buckets - All succed,0.000000,1.000000,0
Even buckets - Half succeed,0.000000,1.000000,0
Scewed towards bottom,0.055145,0.915791,31
Scewed towards top,0.059779,0.908715,31
Random (4+),0.002482,0.996210,6
Random 1,0.006924,0.989426,9
Random 2,0.004032,0.993842,7


In [7]:
/**
 * Luck-meter, as described on FUMBBL: https://fumbbl.com/help:Luck
 * The luck value is a semi-statistical figure that represents the outcome of your rolls compared to the statistically expected average.
 *
 * It produces a value between [0 - 100], where 50% is "expected luck" and 0% is maximally unlucky and 100% is maximally lucky.
 *
 * Luck = Nominator / Denominator
 *
 * **Success**
 * - Nominator is increased by 1 / (pSuccess + pNeutral / 2)
 * - Denominator is increased by 1 / (pSuccess + pNeutral / 2)
 *
 * This will increase the luck value, but never take it above 1.0.
 *
 * **Failure**
 * - Nominator is unchanged.
 * - Denominator is increased by 1 / (pFailure + pNeutral / 2)
 *
 * This will decrease the luck value, but never take it below 0.
 *
 * **Neutral**
 * - Nominator is increased by 1 / (2 * pSuccess + pNeutral)
 * - Denominator is increased by 1 / (2 * pSuccess + pNeutral) + 1 / (2 * pFailure + pNeutral)
 *
 * Advantage:
 * - Tells you something about how lucky or unlucky you are.
 * - Easy to calculate.
 *
 * Critic:
 * - Number cannot be compared between games and players.
 */
val names = mutableListOf<String>()
val nominators = mutableListOf<Double>()
val denominators = mutableListOf<Double>()
val luckValues = mutableListOf<Double>()
distributions.forEach { (name, rolls) ->
    var nominator = 0.0
    var denominator = 0.0
    rolls.forEach { roll ->
        when (roll.isSuccess()) {
            true -> {
                val change = (1.0 / ((roll.successPropability() + roll.neturalPropability()/2.0)) )
                nominator += change
                denominator += change
            }
            false -> {
                denominator += (1.0 / ((roll.failurePropability() + roll.neturalPropability()/2.0)) )
            }
        }
    }
    names.add(name)
    nominators.add(nominator)
    denominators.add(denominator)
    luckValues.add(if (denominator == 0.0) 1.0 else (nominator/denominator))
}
val fumbblLuckValue = dataFrameOf(
    "distribution" to names,
    "nominator" to nominators,
    "denominator" to denominators,
    "luck" to luckValues
)
fumbblLuckValue

distribution,nominator,denominator,luck
All 1's,0.000000,312.200000,0.000000
All 6's,309.000000,309.000000,1.000000
Even buckets - All fail,0.000000,120.000000,0.000000
Even buckets - All succed,120.000000,120.000000,1.000000
Even buckets - Half succeed,120.000000,240.000000,0.500000
Scewed towards bottom,69.300000,243.600000,0.284483
Scewed towards top,166.400000,240.600000,0.691604
Random (4+),118.000000,240.000000,0.491667
Random 1,109.000000,256.700000,0.424620
Random 2,140.600000,274.100000,0.512951


In [8]:
import org.jetbrains.kotlinx.statistics.math3.special.Erf.erf

/***
 * Use the Z-score as a measure of "luck", i.e. how far from the average did we roll. This takes into account the roll
 * and probability for each roll, e.g., it takes into account that rolling a 2+ is easer than a 6+.
 *
 * Definition: Z-score is the number (distance) of standard deviation away (above or below) the mean.
 * By definition the Z score of the mean is 0.
 *
 * This means that the Z-score can say something about how extreme an outcome is within its own distribution,
 * which can be compared to other games or players. The absolute number of dice rolled and success can be different
 * and still have the same Z-score. It just tells you something about how likely the outcome was given the amount of dice
 * rolled and their successes.
 *
 * Advantage:
 * - Standard way to look at probabilities
 * - Can be compared against other players and games
 *
 * Critic:
 * - Z-score is not a [0-1] range, making it harder to interpret.
 * - Z-score is not linear.
 * - If either all succeed or all fail the Z-score is `null`.
 *
 * See https://en.wikipedia.org/wiki/Standard_score
 * See https://en.wikipedia.org/wiki/Bernoulli_trial
 */

/**
 * Wrap Roll results for Bernoulli trial calculations
 * @param p Probability [0-1] that the roll is a success
 * @param success the roll was an actual success
 */
data class Roll(val p: Double, val success: Boolean)

@DataSchema
data class LuckSummary(
    val n: Int,
    val successes: Int,
    val expected: Double,
    val variance: Double,
    val centeredLuck: Double,     // S - μ
    val zScore: Double?,          // (S-μ)/σ, null if σ==0
    val tailP: Double,            // P(X >= S)
    val surprisalLog10: Double,   // -log10(tailP)
    val rarityPercentage: Double? // Two-sided rarity % (assuming normal distribution)
)

@DataSchema
data class LuckRow(
    val distribution: String,
    val summary: LuckSummary
)

/** Exact Poisson–binomial tail P(X >= s) via O(n^2) DP. */
fun tailProbAtLeast(ps: DoubleArray, s: Int): Double {
    val n = ps.size
    var dp = DoubleArray(n + 1) { 0.0 }
    dp[0] = 1.0
    for (p in ps) {
        val next = DoubleArray(n + 1)
        for (k in 0..n) {
            if (dp[k] == 0.0) continue
            next[k] += dp[k] * (1 - p)         // failure
            if (k + 1 <= n) next[k + 1] += dp[k] * p // success
        }
        dp = next
    }
    var tail = 0.0
    for (k in s..n) tail += dp[k]
    return tail.coerceIn(0.0, 1.0)
}

/**
 * One-sided rarity % for a Z-score (negative is "bad luck", positive is "good luck").
 * Note, this assumes a normal distribution, which dice rolls only approximate.
 * It will especially be off at a low amount of rolls
 */
fun rarityPercent(zScore: Double): Double {
    val oneSidedTail = 0.5 * (1 - erf(abs(zScore) / sqrt(2.0))) * 100.0
    return if (zScore >= 0) oneSidedTail else -oneSidedTail
}

fun summarizeLuck(rolls: List<Roll>): LuckSummary {
    val ps = rolls.map { it.p }.toDoubleArray()
    val s = rolls.count { it.success }
    val expected = ps.sum()
    val variance = ps.sumOf { it * (1 - it) }
    val sigma = kotlin.math.sqrt(variance)
    val centered = s - expected
    val z = if (sigma > 0) centered / sigma else null
    val tailP = tailProbAtLeast(ps, s)
    val surprisal = if (tailP > 0) -kotlin.math.log10(tailP) else Double.POSITIVE_INFINITY
    return LuckSummary(
        n = rolls.size,
        successes = s,
        expected = expected,
        variance = variance,
        centeredLuck = centered,
        zScore = z,
        tailP = tailP,
        surprisalLog10 = surprisal,
        rarityPercentage = if (z != null) rarityPercent(z) else null
    )
}

val zScoreLuckSummary = distributions.map { (name, rolls) ->
    val rollsWithProbabilities = rolls.map {
        Roll(it.successPropability(), it.isSuccess())
    }
    LuckRow(name, summarizeLuck(rollsWithProbabilities))
}.toDataFrame().flatten()
zScoreLuckSummary

distribution,n,successes,expected,variance,centeredLuck,zScore,tailP,surprisalLog10,rarityPercentage
All 1's,120,0,58.333333,23.777778,-58.333333,-11.962754,1.000000,-0.000000,-0.000000
All 6's,120,120,62.166667,23.750000,57.833333,11.867150,0.000000,40.819193,0.000000
Even buckets - All fail,120,0,0.000000,0.000000,0.000000,null,1.000000,-0.000000,null
Even buckets - All succed,120,120,120.000000,0.000000,0.000000,null,1.000000,-0.000000,null
Even buckets - Half succeed,120,60,60.000000,30.000000,0.000000,0.000000,0.536342,0.270558,50.000000
Scewed towards bottom,120,39,60.166667,24.083333,-21.166667,-4.313146,0.999996,0.000002,-0.000805
Scewed towards top,120,80,60.500000,23.250000,19.500000,4.044112,0.000035,4.456033,0.002626
Random (4+),120,59,60.000000,30.000000,-1.000000,-0.182574,0.607836,0.216214,-42.756607
Random 1,120,53,62.000000,23.444444,-9.000000,-1.858757,0.975214,0.010900,-3.153082
Random 2,120,62,60.500000,23.194444,1.500000,0.311458,0.417738,0.379096,37.772629


In [9]:
/**
 * Instead of using the Z-Score, we instead describe the "likely outcome" in the given distribution, mapped
 * to [-100 - 100] range.
 *
 * Formally:
 * LuckSym is a linear rescale of the mid-rank CDF of the Poisson–binomial distribution of successes.
 * With -100 being maximally unlucky, 0 being average and +100 being maximally lucky.
 *
 * The interpretation is:
 * - 0: The average outcome
 * - +100: All rolls succeed
 * - -100: All rolls fail
 * - +20:
 *   - 60% of games are ≤ this lucky (so unluckier or equal).
 *   - 40% of games are luckier.
 * - -60:
 *   - 20% of games are ≤ this lucky (so unluckier or equal).
 *   - 80% of games are luckier.
 */

// -------- Exact LuckSym from per-roll p_i and observed successes S --------
fun pbPmf(ps: List<Double>): DoubleArray {
    val n = ps.size
    var dp = DoubleArray(n + 1) { 0.0 }
    dp[0] = 1.0
    for (p in ps) {
        val next = DoubleArray(n + 1)
        for (k in 0..n) {
            val v = dp[k]; if (v == 0.0) continue
            next[k] += v * (1 - p)
            if (k + 1 <= n) next[k + 1] += v * p
        }
        dp = next
    }
    return dp
}

fun midCdf(pmf: DoubleArray, s: Int): Double {
    val ss = s.coerceIn(0, pmf.lastIndex)
    var lt = 0.0
    for (k in 0 until ss) lt += pmf[k]
    return lt + 0.5 * pmf[ss]
}

// ps: Probability array of "success" for each roll
fun luckSymExact(ps: List<Double>, successes: Int): Double {
    // Check for all fail/succeed case, since this will reported wrong otherwise
    val variance = ps.sumOf { it * (1 - it) }
    if (variance == 0.0) {
        return when (successes > 0) {
            true -> 100.0
            false -> -100.0
        }
    }
    val pmf = pbPmf(ps)
    val mid = midCdf(pmf, successes)        // in [0,1]
    return (2.0 * mid - 1.0) * 100.0        // in [-100,+100]
}

val luckSym = distributions.map {
    val title = it.first
    val rolls = it.second
    val ps = rolls.map { it.successPropability() }
    val successes  = rolls.count { it.isSuccess() }
    val luckSym = luckSymExact(ps, successes)
    title to luckSym
}.toDataFrame().rename("first" to "distribution", "second" to "luckSym")
luckSym

distribution,luckSym
All 1's,-100.000000
All 6's,100.000000
Even buckets - All fail,-100.000000
Even buckets - All succed,100.000000
Even buckets - Half succeed,0.000000
Scewed towards bottom,-99.998497
Scewed towards top,99.995101
Random (4+),-14.417840
Random 1,-93.573787
Random 2,24.336664


In [10]:
/**
 * Map luckSym into percentiles [0-100]. This gives the following interpretation:
 *
 * - 50 : Average outcome
 * - 0 : All fails
 * - 100 : All successes
 * - 20 : 80% of outcomes are more lucky than this
 * - 60 : 40% of outcomes are more lucky than this
 */
fun luckSymToPercentile(luckSym: Double): Double =
    ((luckSym + 100.0) / 200.0) * 100.0  // returns 0..100

val luckSymPercentiles = luckSym
    .update("luckSym") { luckSymToPercentile(it as Double) }
    .rename("luckSym").into("luckSymPercentile")
luckSymPercentiles

distribution,luckSymPercentile
All 1's,0.000000
All 6's,100.000000
Even buckets - All fail,0.000000
Even buckets - All succed,100.000000
Even buckets - Half succeed,50.000000
Scewed towards bottom,0.000751
Scewed towards top,99.997551
Random (4+),42.791080
Random 1,3.213106
Random 2,62.168332


In [11]:
/**
 * Compare the relevant results from all the experiments.
 */
listOf(
    jsdValues.select { "distribution" and "jsd"},
    jsdValues.select { "distribution" and "adjustedJSD"},
    fumbblLuckValue.select { "distribution" and "luck" }.rename("luck").into("FFB Luck"),
    zScoreLuckSummary.select { "distribution" and "zScore" },
    zScoreLuckSummary.select { "distribution" and "rarityPercentage" }.rename("rarityPercentage").into("normalDist%"),
    luckSym.select { "distribution" and "luckSym"},
    luckSymPercentiles.select { "distribution" and "luckSymPercentile"}
).reduce { acc, df ->
    acc.join(df) { "distribution" match "distribution" }
}

distribution,jsd,adjustedJSD,FFB Luck,zScore,normalDist%,luckSym,luckSymPercentile
All 1's,0.654858,0.000000,0.000000,-11.962754,-0.000000,-100.000000,0.000000
All 6's,0.654858,0.000000,1.000000,11.867150,0.000000,100.000000,100.000000
Even buckets - All fail,0.000000,1.000000,0.000000,null,null,-100.000000,0.000000
Even buckets - All succed,0.000000,1.000000,1.000000,null,null,100.000000,100.000000
Even buckets - Half succeed,0.000000,1.000000,0.500000,0.000000,50.000000,0.000000,50.000000
Scewed towards bottom,0.055145,0.915791,0.284483,-4.313146,-0.000805,-99.998497,0.000751
Scewed towards top,0.059779,0.908715,0.691604,4.044112,0.002626,99.995101,99.997551
Random (4+),0.002482,0.996210,0.491667,-0.182574,-42.756607,-14.417840,42.791080
Random 1,0.006924,0.989426,0.424620,-1.858757,-3.153082,-93.573787,3.213106
Random 2,0.004032,0.993842,0.512951,0.311458,37.772629,24.336664,62.168332
